# Causal Forests
A causal machine learning method developed by economists, Susan Athey and Stefan Wager.

source: https://towardsdatascience.com/causal-machine-learning-for-econometrics-causal-forests-5ab3aec825a7

A causal forest is built from causal trees, where the causal trees learn a low-dimensional representation of treatment effect heterogeneity. 

### Dataset: Microcredit in Morocco
This tutorial uses a microcredit program dataset from a randomized experiment conducted in rural Moroccan villages in 2006 (Crépon et al., 2015). To understand the impact of microcredit availability, certain villages (treatment group) were given access to microcredit and the amount of loans taken at the household level, was measured in relation to the control group, as an indication of the demand for microcredits. There are observable characteristics for each household, and it is possible to condition on these covariates to discover heterogeneity in the causal effect of microcredit.

In a preprint released earlier this month through the Social Science Research Network (SSRN), Daniel Jacob surveys the machine learning methods available for estimating CATE, using among others, the Moroccan microcredit dataset as an example. Jacob explains that the goal is to find subgroups based on the covariates, where the treatment effect will be different; however, we do not know which subgroups to focus on and there are many covariates, so machine learning is required to estimate the CATE. The following section covers causal inference and the potential outcomes framework to explain how the CATE is calculated.

### Building Causal Forest

In [ ]:
import pandas as pd
from dowhy import CausalModel
import econml
from IPython.display import Image, display

# read Stata .dta file 
df = pd.read_stata("data_rep.dta")

# set variables 
treatment = 'treatment'
outcome = 'loansamt_total'
covariates = ["members_resid_bl", "nadults_resid_bl", "head_age_bl", "act_livestock_bl", "act_business_bl",
              "borrowed_total_bl", "members_resid_d_bl", "nadults_resid_d_bl", "head_age_d_bl",
              "act_livestock_d_bl", "act_business_d_bl", "borrowed_total_d_bl", "ccm_resp_activ",
              "other_resp_activ", "ccm_resp_activ_d", "other_resp_activ_d", "head_educ_1", "nmember_age6_16"]

# build causal graph with dowhy 
model = CausalModel(
    data=df,
    treatment=treatment, 
    outcome=outcome, 
    common_causes=covariates, 
    instruments=None, 
    effect_modifiers=None)
model.view_model()
#display(Image(filename="causal_model.png"))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# drop missing data 
all_variables = [treatment]+[outcome]+covariates
df = df.dropna(axis=0, subset=all_variables)
print(df.shape)
# split data into train and test sets 
train, test = train_test_split(df, test_size=0.2)

# set variables for causal forest Y=outcome, T=treatment, X=covariates, W=effect_modifiers 
Y = train[outcome]
T = train[treatment]
X = train[covariates]
W = None
X_test = test[covariates]

In [ ]:
from econml.dml import CausalForestDML
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.linear_model import LassoCV
from econml.sklearn_extensions.linear_model import DebiasedLasso


# set parameters for causal forest 
causal_forest = CausalForestDML(criterion='het', 
                                n_estimators=10000,       
                                min_samples_leaf=10, 
                                max_depth=None, 
                                max_samples=0.5,
                                discrete_treatment=False,
                                honest=True,
                                inference=True,
                                cv=10,
                                model_t=LassoCV(), 
                                model_y=LassoCV(),
                                )
                      
# fit train data to causal forest model 
causal_forest.fit(Y, T, X=X, W=W)
# estimate the CATE with the test set 
causal_forest.const_marginal_ate(X_test)

The ground truth average CATE was reported to be around 1300. The causal forest built here will return CATE values between 1100 and 1200.

In [ ]:
import shap
from econml.dml import CausalForestDML

# fit causal forest with default parameters 
causal_forest = CausalForestDML()
causal_forest.fit(Y, T, X=X, W=W)

In [ ]:
# calculate shap values of causal forest model 
shap_values = causal_forest.shap_values(X)
# plot shap values 
#print(shap_values['loansamt_total']['treatment'])
shap.summary_plot(shap_values['loansamt_total']['treatment'])

The feature (covariate) that had the greatest importance was the age of the head of the household, followed by the number of children aged 6 to 16 in the household. It appears that household age featured importantly when the average causal effect was negative. This helps identify subgroups, for example, feature importance suggests that older and younger households should be evaluated separately to estimate the CATE.

In [ ]:

# use causal forest model to estimate treatment effects  
treatment_effects = causal_forest.effect(X)
# calculate lower bound and upper bound confidence intervals 
lb, ub = causal_forest.effect_interval(X, alpha=0.05)

# convert arrays to pandas dataframes for plotting
te_df = pd.DataFrame(treatment_effects, columns=['cate'])
lb_df = pd.DataFrame(lb, columns=['lb'])
ub_df = pd.DataFrame(ub, columns=['ub'])

# merge dataframes and sort 
df = pd.concat([te_df, lb_df, ub_df], axis =1)
#df = te_df
df.sort_values('cate', inplace=True, ascending=True)
df.reset_index(inplace=True, drop=True)

# calculate rolling mean
z = df.rolling(window=30, center=True).mean()

In [ ]:

import matplotlib.pyplot as plt

# set plot size
fig, ax = plt.subplots(figsize=(12, 8))
# plot lines for treatment effects and confidence intervals
ax.plot(z['cate'],
        marker='.', linestyle='-', linewidth=0.5, label='CATE', color='indigo')
ax.plot(z['lb'],
        marker='.', linestyle='-', linewidth=0.5, color='steelblue')
ax.plot(z['ub'],
        marker='.', linestyle='-', linewidth=0.5, color='steelblue')
# label axes and create legend
ax.set_ylabel('Treatment Effects')
ax.set_xlabel('Number of observations')
ax.legend()